# Erasmus Neural Networks
http://michalbereta.pl/nn
## Introduction to TensorFlow and Keras


## Before you start

Exacute the examples.

Then, do the tasks and send back the notebook.

Change the name of this notebook according to the schema: {YourSurname}\_{YourFirstName}\_{OriginalFileName}.

Be sure to fill all places with "YOUR ANSWER HERE".

When ready, send the notebook, with all the necessary files zipped, to the teacher.

### TensorFlow and Keras

Keras: https://keras.io/

TensorFlow: https://www.tensorflow.org/

### Checking the configuration of TensorFlow

In [59]:
import tensorflow as tf


print(tf.__version__ )
print("------------------")
if tf.test.gpu_device_name():
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
else:
    print("Please install GPU version of TF")

print("------------------")
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())
print("------------------")
print("Is built with cuda:",tf.test.is_built_with_cuda())
print("------------------")

with tf.device('/gpu:0'):
    a = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[2, 3], name='a')
    b = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[3, 2], name='b')
    c = tf.matmul(a, b)

with tf.Session() as sess:
    print (sess.run(c))
    
    

1.15.0
------------------
Default GPU Device: /device:GPU:0
------------------
[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 17915482826514673690
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 3009832236519094315
physical_device_desc: "device: XLA_CPU device"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 12205076489077683280
physical_device_desc: "device: XLA_GPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 14912199066
locality {
  bus_id: 1
  links {
  }
}
incarnation: 7571529251576891155
physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5"
]
------------------
Is built with cuda: True
------------------
[[22. 28.]
 [49. 64.]]


### Solving simple optimization problem in TensorFlow

In the following example we are going to find $w$ so that 

$y = w*x$

for $y=12.0$ and $x=1.1$. 

We will minimie the squared error.

In [60]:
import tensorflow as tf

x = tf.constant(1.1, name='input')
w = tf.Variable(0.8, name='weight') #random starting value
y = tf.multiply(w, x, name='output')
y_ = tf.constant(12.0, name='correct_value')

loss = tf.pow(y - y_, 2, name='loss')
train_step = tf.train.GradientDescentOptimizer(0.05).minimize(loss)

for value in [x, w, y, y_, loss]:
    tf.summary.scalar(value.op.name, value)

summaries = tf.summary.merge_all()

sess = tf.Session()
summary_writer = tf.summary.FileWriter('log_simple_stats', sess.graph)

sess.run(tf.initialize_all_variables())
for i in range(300):
    summary_writer.add_summary(sess.run(summaries), i)
    sess.run(train_step)
    print(sess.run([loss, w, y]))


[95.54046, 2.0232, 2.2255201]
[73.81847, 3.098393, 3.4082325]
[57.035175, 4.0434875, 4.4478364]
[44.067715, 4.8742256, 5.361648]
[34.048523, 5.6044445, 6.164889]
[26.30728, 6.246307, 6.870938]
[20.326078, 6.810504, 7.4915547]
[15.704759, 7.306433, 8.037077]
[12.134145, 7.742355, 8.51659]
[9.375332, 8.12553, 8.938084]
[7.2437654, 8.462341, 9.308576]
[5.5968285, 8.758398, 9.634238]
[4.324341, 9.018632, 9.920495]
[3.3411624, 9.247377, 10.172115]
[2.5815215, 9.448444, 10.393289]
[1.9945862, 9.625183, 10.587702]
[1.5410994, 9.780536, 10.75859]
[1.1907172, 9.91709, 10.9088]
[0.91999674, 10.037123, 11.040835]
[0.7108282, 10.142631, 11.156894]
[0.5492141, 10.235373, 11.25891]
[0.42434508, 10.316893, 11.348582]
[0.32786572, 10.388549, 11.427404]
[0.25332308, 10.451534, 11.496688]
[0.19572787, 10.506899, 11.557589]
[0.15122747, 10.555564, 11.61112]
[0.11684401, 10.598341, 11.658175]
[0.090278424, 10.6359415, 11.699536]
[0.069753386, 10.668992, 11.735891]
[0.053894546, 10.698044, 11.767848]
[0.04

### Solving XOR with Keras and TensorFlow

In [61]:
import tensorflow as tf
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.models import load_model
from keras.utils import np_utils


print('tensorflow version',tf.__version__)
print('keras version',keras.__version__)

xtrain = np.array([[0,0],[0,1],[1,0],[1,1]])
ytrain = np.array([0,1,1,0])

inputs_num = 2

# one hot encode outputs
ytrain = np_utils.to_categorical(ytrain)
print('\nytrain=',ytrain)
num_classes = ytrain.shape[1]

# define baseline model
model = Sequential()
model.add(Dense(2, input_dim=inputs_num, kernel_initializer='normal', activation='relu'))
model.add(Dense(num_classes, kernel_initializer='normal', activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

print()
# Fit the model
model.fit(xtrain, ytrain, epochs=100, batch_size=4, verbose=2)

# Final evaluation of the model
scores = model.evaluate(xtrain, ytrain, verbose=0)
print("\nTest error: %.2f%%" % (100-scores[1]*100))

predictions = model.predict(xtrain)
print('\npredictions',predictions)
predicted_classes = model.predict_classes(xtrain)
print('\npredicted_classes',predicted_classes)


print('end')

tensorflow version 1.15.0
keras version 2.2.5

ytrain= [[1. 0.]
 [0. 1.]
 [0. 1.]
 [1. 0.]]

Epoch 1/100
 - 2s - loss: 0.6932 - acc: 0.5000
Epoch 2/100
 - 0s - loss: 0.6932 - acc: 0.5000
Epoch 3/100
 - 0s - loss: 0.6932 - acc: 0.5000
Epoch 4/100
 - 0s - loss: 0.6932 - acc: 0.7500
Epoch 5/100
 - 0s - loss: 0.6931 - acc: 0.7500
Epoch 6/100
 - 0s - loss: 0.6931 - acc: 0.7500
Epoch 7/100
 - 0s - loss: 0.6931 - acc: 0.5000
Epoch 8/100
 - 0s - loss: 0.6931 - acc: 0.5000
Epoch 9/100
 - 0s - loss: 0.6930 - acc: 0.5000
Epoch 10/100
 - 0s - loss: 0.6930 - acc: 0.5000
Epoch 11/100
 - 0s - loss: 0.6930 - acc: 0.5000
Epoch 12/100
 - 0s - loss: 0.6930 - acc: 0.5000
Epoch 13/100
 - 0s - loss: 0.6929 - acc: 0.5000
Epoch 14/100
 - 0s - loss: 0.6929 - acc: 0.5000
Epoch 15/100
 - 0s - loss: 0.6929 - acc: 0.5000
Epoch 16/100
 - 0s - loss: 0.6929 - acc: 0.5000
Epoch 17/100
 - 0s - loss: 0.6929 - acc: 0.5000
Epoch 18/100
 - 0s - loss: 0.6929 - acc: 0.5000
Epoch 19/100
 - 0s - loss: 0.6928 - acc: 0.7500
Epoc

### 1D function approximation with Keras and TensorFlow

In [62]:
import tensorflow as tf
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.models import load_model
from keras.utils import np_utils
import matplotlib.pyplot as plt

%matplotlib notebook

#####################################################
def fun1(x):
    return np.sin(0.5*x) + np.sin(x + 2)
#####################################################

print('tensorflow version',tf.__version__)
print('keras version',keras.__version__)

xmin = -6
xmax = 6

x = np.arange(xmin, xmax, 0.2)
#print (x)

#real values of unknown process
d = fun1(x)

#training data with noise
sigma = 0.1
tr_d = d + np.random.randn(len(d)) * sigma

plt.plot(x, d)
plt.plot(x, tr_d, 'o')

plt.show()

inputs_num = 1
outputs_num = 1

# define baseline model
model = Sequential()
model.add(Dense(50, input_dim=inputs_num, kernel_initializer='normal', activation='relu'))
model.add(Dense(outputs_num, kernel_initializer='normal', activation='linear'))

#model.compile(loss='mean_squared_error', optimizer=keras.optimizers.SGD(lr=0.001))
model.compile(loss='mean_squared_error', optimizer=keras.optimizers.Adam())

print()
# Fit the model
model.fit(x, tr_d, epochs=2000, verbose=1)

# Final evaluation of the model
train_ans = model.predict(x)
#print('train_ans',train_ans)

#test data
xtest = np.arange(xmin, xmax, 0.02) 
ytest = model.predict(xtest)

plt.figure()
plt.plot(x, tr_d, 'o')
plt.plot(xtest, ytest)
plt.show()
print('end')

tensorflow version 1.15.0
keras version 2.2.5


<IPython.core.display.Javascript object>


Epoch 1/2000
60/60 [==============================] - 2s 41ms/step - loss: 1.0277
Epoch 2/2000
60/60 [==============================] - 0s 185us/step - loss: 1.0107
Epoch 3/2000
60/60 [==============================] - 0s 185us/step - loss: 0.9916
Epoch 4/2000
60/60 [==============================] - 0s 183us/step - loss: 0.9758
Epoch 5/2000
60/60 [==============================] - 0s 178us/step - loss: 0.9569
Epoch 6/2000
60/60 [==============================] - 0s 233us/step - loss: 0.9408
Epoch 7/2000
60/60 [==============================] - 0s 181us/step - loss: 0.9237
Epoch 8/2000
60/60 [==============================] - 0s 178us/step - loss: 0.9080
Epoch 9/2000
60/60 [==============================] - 0s 169us/step - loss: 0.8931
Epoch 10/2000
60/60 [==============================] - 0s 157us/step - loss: 0.8769
Epoch 11/2000
60/60 [==============================] - 0s 173us/step - loss: 0.8617
Epoch 12/2000
60/60 [==============================] - 0s 145us/step - loss: 0.8446
E

<IPython.core.display.Javascript object>

end


### Task 1


Use Keras to train MLP with 1 and 2 hidden layers for pima and sonar databases. Compare two types of output layers: 1) linear with MSE loss, 2) softmax with cross-entropy loss.

#### YOUR ANSWERS HERE
My results are as follows...

# **Pima-diabetes**

## **MLP1HL (Pima-diabetes)**

### **Linear with MSE loss**

In [63]:
%matplotlib notebook
import tensorflow as tf
import keras as krs
import numpy as np
import pandas as pd

X = pd.read_csv('pima-diabetes.csv')

y = X.iloc[:,-1].astype('int32')
X = X.iloc[:,:-1].astype(np.float)

from sklearn.model_selection import train_test_split

xtrain, xtest, ytrain, ytest = train_test_split(X, y, test_size=0.25, random_state=0)

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.models import load_model
from keras.utils import np_utils

seed = 12345
numpy.random.seed(seed)

inputs_num = xtrain.shape[1]

ytrain = np_utils.to_categorical(ytrain)
ytest = np_utils.to_categorical(ytest)
num_classes = ytest.shape[1]
print(ytest.shape)

## Create our model
model = Sequential()

# 1st layer: RELU
model.add(Dense(500, input_dim=inputs_num, kernel_initializer='normal', activation='relu'))
# output layer: activation linear
model.add(Dense(num_classes, kernel_initializer='normal', activation='linear'))

# Compile the model: MSE
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])

logger = keras.callbacks.ModelCheckpoint('pima_model_MLP1HL_linear_mse.hdf5', monitor='val_acc', verbose=0, save_best_only=True)

model.fit(xtrain, ytrain, validation_data=(xtest, ytest), epochs=100, batch_size=200, verbose=2, callbacks=[logger])

scores = model.evaluate(xtest, ytest, verbose=0)
print("Test error: %.2f%%" % (100-scores[1]*100))

#reading the best model from file
model2 = load_model('pima_model_MLP1HL_linear_mse.hdf5')
scores2 = model2.evaluate(xtest, ytest, batch_size=200)
print('network from file:')
print("Test error: %.2f%%" % (100-scores2[1]*100))

(192, 2)
Train on 575 samples, validate on 192 samples
Epoch 1/2000
 - 3s - loss: 23.0383 - acc: 0.6383 - val_loss: 18.1302 - val_acc: 0.6875
Epoch 2/2000
 - 0s - loss: 13.7840 - acc: 0.6087 - val_loss: 3.0818 - val_acc: 0.5469
Epoch 3/2000
 - 0s - loss: 4.2932 - acc: 0.4870 - val_loss: 8.9009 - val_acc: 0.4427
Epoch 4/2000
 - 0s - loss: 7.4165 - acc: 0.4939 - val_loss: 1.8965 - val_acc: 0.6042
Epoch 5/2000
 - 0s - loss: 1.6209 - acc: 0.6087 - val_loss: 4.0334 - val_acc: 0.6042
Epoch 6/2000
 - 0s - loss: 3.9330 - acc: 0.6035 - val_loss: 2.4644 - val_acc: 0.6094
Epoch 7/2000
 - 0s - loss: 1.2932 - acc: 0.5896 - val_loss: 1.3675 - val_acc: 0.5625
Epoch 8/2000
 - 0s - loss: 1.9645 - acc: 0.5183 - val_loss: 1.7500 - val_acc: 0.4167
Epoch 9/2000
 - 0s - loss: 1.2293 - acc: 0.4452 - val_loss: 0.8048 - val_acc: 0.4844
Epoch 10/2000
 - 0s - loss: 0.9175 - acc: 0.5357 - val_loss: 1.2448 - val_acc: 0.6042
Epoch 11/2000
 - 0s - loss: 0.7791 - acc: 0.6435 - val_loss: 0.4936 - val_acc: 0.6562
Epoch

### **Softmax with cross-entropy loss**

In [64]:
%matplotlib notebook
import tensorflow as tf
import keras as krs
import numpy as np
import pandas as pd

X = pd.read_csv('pima-diabetes.csv')

y = X.iloc[:,-1].astype('int32')
X = X.iloc[:,:-1].astype(np.float)

from sklearn.model_selection import train_test_split

xtrain, xtest, ytrain, ytest = train_test_split(X, y, test_size=0.25, random_state=0)

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.models import load_model
from keras.utils import np_utils

seed = 12345
numpy.random.seed(seed)

inputs_num = xtrain.shape[1]

ytrain = np_utils.to_categorical(ytrain)
ytest = np_utils.to_categorical(ytest)
num_classes = ytest.shape[1]
print(ytest.shape)

## Create our model
model = Sequential()

# 1st layer: RELU
model.add(Dense(500, input_dim=inputs_num, kernel_initializer='normal', activation='relu'))
# output layer: activation softmax
model.add(Dense(num_classes, kernel_initializer='normal', activation='softmax'))

# Compile the model: loss-crossentropy
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

logger = keras.callbacks.ModelCheckpoint('pima_model_MLP1HL_softmax_cross.hdf5', monitor='val_acc', verbose=0, save_best_only=True)

model.fit(xtrain, ytrain, validation_data=(xtest, ytest), epochs=100, batch_size=200, verbose=2, callbacks=[logger])

scores = model.evaluate(xtest, ytest, verbose=0)
print("Test error: %.2f%%" % (100-scores[1]*100))

#reading the best model from file
model2 = load_model('pima_model_MLP1HL_softmax_cross.hdf5')
scores2 = model2.evaluate(xtest, ytest, batch_size=200)
print('network from file:')
print("Test error: %.2f%%" % (100-scores2[1]*100))

(192, 2)
Train on 575 samples, validate on 192 samples
Epoch 1/2000
 - 3s - loss: 1.5142 - acc: 0.6104 - val_loss: 1.6673 - val_acc: 0.7083
Epoch 2/2000
 - 0s - loss: 1.5015 - acc: 0.5357 - val_loss: 0.9608 - val_acc: 0.5365
Epoch 3/2000
 - 0s - loss: 1.0092 - acc: 0.5965 - val_loss: 0.9527 - val_acc: 0.6719
Epoch 4/2000
 - 0s - loss: 0.8677 - acc: 0.6643 - val_loss: 0.9315 - val_acc: 0.5469
Epoch 5/2000
 - 0s - loss: 0.7880 - acc: 0.5739 - val_loss: 0.8852 - val_acc: 0.7031
Epoch 6/2000
 - 0s - loss: 0.8438 - acc: 0.6661 - val_loss: 0.7555 - val_acc: 0.5833
Epoch 7/2000
 - 0s - loss: 0.7522 - acc: 0.5478 - val_loss: 0.6166 - val_acc: 0.7083
Epoch 8/2000
 - 0s - loss: 0.7227 - acc: 0.6591 - val_loss: 0.6106 - val_acc: 0.6927
Epoch 9/2000
 - 0s - loss: 0.6277 - acc: 0.6783 - val_loss: 0.6055 - val_acc: 0.6771
Epoch 10/2000
 - 0s - loss: 0.6498 - acc: 0.6678 - val_loss: 0.6279 - val_acc: 0.6979
Epoch 11/2000
 - 0s - loss: 0.6171 - acc: 0.6643 - val_loss: 0.6002 - val_acc: 0.6823
Epoch 12

## **MLP2HL (Pima-diabetes)**

### **Linear with MSE loss**

In [65]:
%matplotlib notebook
import tensorflow as tf
import keras as krs
import numpy as np
import pandas as pd

X = pd.read_csv('pima-diabetes.csv')

y = X.iloc[:,-1].astype('int32')
X = X.iloc[:,:-1].astype(np.float)

from sklearn.model_selection import train_test_split

xtrain, xtest, ytrain, ytest = train_test_split(X, y, test_size=0.25, random_state=0)

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.models import load_model
from keras.utils import np_utils

seed = 12345
numpy.random.seed(seed)

inputs_num = xtrain.shape[1]

ytrain = np_utils.to_categorical(ytrain)
ytest = np_utils.to_categorical(ytest)
num_classes = ytest.shape[1]
print(ytest.shape)

## Create our model
model = Sequential()

# 1st layer: RELU
model.add(Dense(500, input_dim=inputs_num, kernel_initializer='normal', activation='relu'))
# 2nd layer: sigmoid
model.add(Dense(100, input_dim=inputs_num, kernel_initializer='normal', activation='sigmoid'))
# output layer: activation linear
model.add(Dense(num_classes, kernel_initializer='normal', activation='linear'))

# Compile the model: MSE
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])

logger = keras.callbacks.ModelCheckpoint('pima_model_MLP2HL_linear_mse.hdf5', monitor='val_acc', verbose=0, save_best_only=True)

model.fit(xtrain, ytrain, validation_data=(xtest, ytest), epochs=100, batch_size=200, verbose=2, callbacks=[logger])

scores = model.evaluate(xtest, ytest, verbose=0)
print("Test error: %.2f%%" % (100-scores[1]*100))

#reading the best model from file
model2 = load_model('pima_model_MLP2HL_linear_mse.hdf5')
scores2 = model2.evaluate(xtest, ytest, batch_size=200)
print('network from file:')
print("Test error: %.2f%%" % (100-scores2[1]*100))

(192, 2)
Train on 575 samples, validate on 192 samples
Epoch 1/2000
 - 3s - loss: 0.3808 - acc: 0.6365 - val_loss: 0.2439 - val_acc: 0.6875
Epoch 2/2000
 - 0s - loss: 0.2596 - acc: 0.6348 - val_loss: 0.2115 - val_acc: 0.6979
Epoch 3/2000
 - 0s - loss: 0.2261 - acc: 0.6417 - val_loss: 0.1958 - val_acc: 0.6979
Epoch 4/2000
 - 0s - loss: 0.2170 - acc: 0.6574 - val_loss: 0.1945 - val_acc: 0.7031
Epoch 5/2000
 - 0s - loss: 0.2122 - acc: 0.6643 - val_loss: 0.1968 - val_acc: 0.7500
Epoch 6/2000
 - 0s - loss: 0.2109 - acc: 0.6609 - val_loss: 0.1930 - val_acc: 0.6875
Epoch 7/2000
 - 0s - loss: 0.2053 - acc: 0.6609 - val_loss: 0.1897 - val_acc: 0.7188
Epoch 8/2000
 - 0s - loss: 0.2044 - acc: 0.6939 - val_loss: 0.1866 - val_acc: 0.7396
Epoch 9/2000
 - 0s - loss: 0.2026 - acc: 0.6904 - val_loss: 0.1877 - val_acc: 0.7448
Epoch 10/2000
 - 0s - loss: 0.2017 - acc: 0.7009 - val_loss: 0.1863 - val_acc: 0.7500
Epoch 11/2000
 - 0s - loss: 0.2031 - acc: 0.6852 - val_loss: 0.1870 - val_acc: 0.7552
Epoch 12

### **Softmax with cross-entropy loss**

In [66]:
%matplotlib notebook
import tensorflow as tf
import keras as krs
import numpy as np
import pandas as pd

X = pd.read_csv('pima-diabetes.csv')

y = X.iloc[:,-1].astype('int32')
X = X.iloc[:,:-1].astype(np.float)

from sklearn.model_selection import train_test_split

xtrain, xtest, ytrain, ytest = train_test_split(X, y, test_size=0.25, random_state=0)

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.models import load_model
from keras.utils import np_utils

seed = 12345
numpy.random.seed(seed)

inputs_num = xtrain.shape[1]

ytrain = np_utils.to_categorical(ytrain)
ytest = np_utils.to_categorical(ytest)
num_classes = ytest.shape[1]
print(ytest.shape)

## Create our model
model = Sequential()

# 1st layer: RELU
model.add(Dense(500, input_dim=inputs_num, kernel_initializer='normal', activation='relu'))
# 2nd layer: sigmoid
model.add(Dense(100, input_dim=inputs_num, kernel_initializer='normal', activation='sigmoid'))
# output layer: activation softmax
model.add(Dense(num_classes, kernel_initializer='normal', activation='softmax'))

# Compile the model: loss-crossentropy
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

logger = keras.callbacks.ModelCheckpoint('pima_model_MLP2HL_softmax_cross.hdf5', monitor='val_acc', verbose=0, save_best_only=True)

model.fit(xtrain, ytrain, validation_data=(xtest, ytest), epochs=100, batch_size=200, verbose=2, callbacks=[logger])

scores = model.evaluate(xtest, ytest, verbose=0)
print("Test error: %.2f%%" % (100-scores[1]*100))

#reading the best model from file
model2 = load_model('pima_model_MLP2HL_softmax_cross.hdf5')
scores2 = model2.evaluate(xtest, ytest, batch_size=200)
print('network from file:')
print("Test error: %.2f%%" % (100-scores2[1]*100))

(192, 2)
Train on 575 samples, validate on 192 samples
Epoch 1/2000
 - 4s - loss: 0.6612 - acc: 0.6330 - val_loss: 0.5944 - val_acc: 0.6823
Epoch 2/2000
 - 0s - loss: 0.6285 - acc: 0.6417 - val_loss: 0.5973 - val_acc: 0.6823
Epoch 3/2000
 - 0s - loss: 0.6198 - acc: 0.6452 - val_loss: 0.5742 - val_acc: 0.6823
Epoch 4/2000
 - 0s - loss: 0.6133 - acc: 0.6400 - val_loss: 0.5747 - val_acc: 0.7031
Epoch 5/2000
 - 0s - loss: 0.6016 - acc: 0.6626 - val_loss: 0.5623 - val_acc: 0.7083
Epoch 6/2000
 - 0s - loss: 0.5982 - acc: 0.6661 - val_loss: 0.5591 - val_acc: 0.7448
Epoch 7/2000
 - 0s - loss: 0.5884 - acc: 0.6887 - val_loss: 0.5553 - val_acc: 0.7344
Epoch 8/2000
 - 0s - loss: 0.5879 - acc: 0.6835 - val_loss: 0.5472 - val_acc: 0.7344
Epoch 9/2000
 - 0s - loss: 0.5801 - acc: 0.6939 - val_loss: 0.5528 - val_acc: 0.7656
Epoch 10/2000
 - 0s - loss: 0.5772 - acc: 0.7026 - val_loss: 0.5415 - val_acc: 0.7344
Epoch 11/2000
 - 0s - loss: 0.5808 - acc: 0.6817 - val_loss: 0.5552 - val_acc: 0.7604
Epoch 12

# **Pima-diabetes: Comparing the results**

In [67]:
import tensorflow as tf
import numpy as np
import keras
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.models import load_model
from keras.utils import np_utils

X = pd.read_csv('pima-diabetes.csv')

y = X.iloc[:,-1].astype('int32')
X = X.iloc[:,:-1].astype(np.float)

from sklearn.model_selection import train_test_split

xtrain, xtest, ytrain, ytest = train_test_split(X, y, test_size=0.25, random_state=0)

# one hot encode outputs
ytrain = np_utils.to_categorical(ytrain)
ytest = np_utils.to_categorical(ytest)

#Compare the networks

#MLP1HL (Linear with MSE loss)
model = load_model('pima_model_MLP1HL_linear_mse.hdf5')
scores = model.evaluate(xtest, ytest, batch_size=200)
print("Test error (Model MLP1HL (Linear with MSE loss)): %.2f%%" % (100-scores[1]*100))

#MLP1HL (Softmax with cross-entropy loss)
model = load_model('pima_model_MLP1HL_softmax_cross.hdf5')
scores = model.evaluate(xtest, ytest, batch_size=200)
print("Test error (Model MLP1HL (Softmax with cross-entropy loss)): %.2f%%" % (100-scores[1]*100))

#MLP2HL (Linear with MSE loss)
model = load_model('pima_model_MLP2HL_linear_mse.hdf5')
scores = model.evaluate(xtest, ytest, batch_size=200)
print("Test error (Model MLP2HL (Linear with MSE loss)): %.2f%%" % (100-scores[1]*100))

#MLP2HL (Softmax with cross-entropy loss)
model = load_model('pima_model_MLP2HL_softmax_cross.hdf5')
scores = model.evaluate(xtest, ytest, batch_size=200)
print("Test error (Model MLP2HL (Softmax with cross-entropy loss)): %.2f%%" % (100-scores[1]*100))

print('end')

192/192 [==============================] - 1s 7ms/step
Test error (Model MLP1HL (Linear with MSE loss)): 22.40%
192/192 [==============================] - 1s 7ms/step
Test error (Model MLP1HL (Softmax with cross-entropy loss)): 20.83%
192/192 [==============================] - 1s 7ms/step
Test error (Model MLP2HL (Linear with MSE loss)): 19.79%
192/192 [==============================] - 1s 7ms/step
Test error (Model MLP2HL (Softmax with cross-entropy loss)): 19.79%
end


# **Sonar**

## **MLP1HL (Sonar)**

### **Linear with MSE loss**

In [73]:
%matplotlib notebook
import tensorflow as tf
import keras as krs
import numpy as np
import pandas as pd

X = pd.read_csv('sonar.csv')

y = X.iloc[:,-1].astype('U20')
X = X.iloc[:,:-1].astype('float')

for i in range(len(y)):
    if y[i]=='Rock':
        y[i] = 0
    elif y[i]=='Mine':
        y[i] = 1
        
y = y.astype(int)

from sklearn.model_selection import train_test_split

xtrain, xtest, ytrain, ytest = train_test_split(X, y, test_size=0.25, random_state=0)

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.models import load_model
from keras.utils import np_utils

seed = 12345
numpy.random.seed(seed)

inputs_num = xtrain.shape[1]

ytrain = np_utils.to_categorical(ytrain)
ytest = np_utils.to_categorical(ytest)
num_classes = ytest.shape[1]
print(ytest.shape)

## Create our model
model = Sequential()

# 1st layer: RELU
model.add(Dense(500, input_dim=inputs_num, kernel_initializer='normal', activation='relu'))
# output layer: activation linear
model.add(Dense(num_classes, kernel_initializer='normal', activation='linear'))

# Compile the model: MSE
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])

logger = keras.callbacks.ModelCheckpoint('sonar_model_MLP1HL_linear_mse.hdf5', monitor='val_acc', verbose=0, save_best_only=True)

model.fit(xtrain, ytrain, validation_data=(xtest, ytest), epochs=100, batch_size=200, verbose=2, callbacks=[logger])

scores = model.evaluate(xtest, ytest, verbose=0)
print("Test error: %.2f%%" % (100-scores[1]*100))

#reading the best model from file
model2 = load_model('sonar_model_MLP1HL_linear_mse.hdf5')
scores2 = model2.evaluate(xtest, ytest, batch_size=200)
print('network from file:')
print("Test error: %.2f%%" % (100-scores2[1]*100))

(52, 2)
Train on 155 samples, validate on 52 samples
Epoch 1/100
 - 5s - loss: 0.4776 - acc: 0.4452 - val_loss: 0.3864 - val_acc: 0.4615
Epoch 2/100
 - 0s - loss: 0.3868 - acc: 0.4387 - val_loss: 0.3103 - val_acc: 0.4231
Epoch 3/100
 - 0s - loss: 0.3107 - acc: 0.4387 - val_loss: 0.2640 - val_acc: 0.4808
Epoch 4/100
 - 0s - loss: 0.2614 - acc: 0.5935 - val_loss: 0.2499 - val_acc: 0.5769
Epoch 5/100
 - 0s - loss: 0.2409 - acc: 0.7290 - val_loss: 0.2615 - val_acc: 0.5769
Epoch 6/100
 - 0s - loss: 0.2437 - acc: 0.6581 - val_loss: 0.2831 - val_acc: 0.5769
Epoch 7/100
 - 0s - loss: 0.2560 - acc: 0.6194 - val_loss: 0.2985 - val_acc: 0.5577
Epoch 8/100
 - 0s - loss: 0.2641 - acc: 0.5935 - val_loss: 0.3014 - val_acc: 0.5577
Epoch 9/100
 - 0s - loss: 0.2624 - acc: 0.6000 - val_loss: 0.2935 - val_acc: 0.5577
Epoch 10/100
 - 0s - loss: 0.2522 - acc: 0.6129 - val_loss: 0.2797 - val_acc: 0.5769
Epoch 11/100
 - 0s - loss: 0.2378 - acc: 0.6387 - val_loss: 0.2647 - val_acc: 0.5769
Epoch 12/100
 - 0s - 

### **Softmax with cross-entropy loss**

In [74]:
%matplotlib notebook
import tensorflow as tf
import keras as krs
import numpy as np
import pandas as pd

X = pd.read_csv('sonar.csv')

y = X.iloc[:,-1].astype('U20')
X = X.iloc[:,:-1].astype('float')

for i in range(len(y)):
    if y[i]=='Rock':
        y[i] = 0
    elif y[i]=='Mine':
        y[i] = 1
        
y = y.astype(int)

from sklearn.model_selection import train_test_split

xtrain, xtest, ytrain, ytest = train_test_split(X, y, test_size=0.25, random_state=0)

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.models import load_model
from keras.utils import np_utils

seed = 12345
numpy.random.seed(seed)

inputs_num = xtrain.shape[1]

ytrain = np_utils.to_categorical(ytrain)
ytest = np_utils.to_categorical(ytest)
num_classes = ytest.shape[1]
print(ytest.shape)

## Create our model
model = Sequential()

# 1st layer: RELU
model.add(Dense(500, input_dim=inputs_num, kernel_initializer='normal', activation='relu'))
# output layer: activation softmax
model.add(Dense(num_classes, kernel_initializer='normal', activation='softmax'))

# Compile the model: loss-crossentropy
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

logger = keras.callbacks.ModelCheckpoint('sonar_model_MLP1HL_softmax_cross.hdf5', monitor='val_acc', verbose=0, save_best_only=True)

model.fit(xtrain, ytrain, validation_data=(xtest, ytest), epochs=100, batch_size=200, verbose=2, callbacks=[logger])

scores = model.evaluate(xtest, ytest, verbose=0)
print("Test error: %.2f%%" % (100-scores[1]*100))

#reading the best model from file
model2 = load_model('sonar_model_MLP1HL_softmax_cross.hdf5')
scores2 = model2.evaluate(xtest, ytest, batch_size=200)
print('network from file:')
print("Test error: %.2f%%" % (100-scores2[1]*100))

(52, 2)
Train on 155 samples, validate on 52 samples
Epoch 1/100
 - 5s - loss: 0.7055 - acc: 0.4452 - val_loss: 0.6936 - val_acc: 0.4615
Epoch 2/100
 - 0s - loss: 0.6864 - acc: 0.6194 - val_loss: 0.6888 - val_acc: 0.5577
Epoch 3/100
 - 0s - loss: 0.6730 - acc: 0.5677 - val_loss: 0.6895 - val_acc: 0.5577
Epoch 4/100
 - 0s - loss: 0.6648 - acc: 0.5613 - val_loss: 0.6905 - val_acc: 0.5577
Epoch 5/100
 - 0s - loss: 0.6575 - acc: 0.5613 - val_loss: 0.6885 - val_acc: 0.5577
Epoch 6/100
 - 0s - loss: 0.6490 - acc: 0.5742 - val_loss: 0.6834 - val_acc: 0.5577
Epoch 7/100
 - 0s - loss: 0.6390 - acc: 0.6065 - val_loss: 0.6766 - val_acc: 0.5769
Epoch 8/100
 - 0s - loss: 0.6286 - acc: 0.6774 - val_loss: 0.6696 - val_acc: 0.5577
Epoch 9/100
 - 0s - loss: 0.6190 - acc: 0.7097 - val_loss: 0.6635 - val_acc: 0.5577
Epoch 10/100
 - 0s - loss: 0.6108 - acc: 0.6903 - val_loss: 0.6588 - val_acc: 0.5577
Epoch 11/100
 - 0s - loss: 0.6037 - acc: 0.7097 - val_loss: 0.6551 - val_acc: 0.5577
Epoch 12/100
 - 0s - 

## **MLP2HL (Sonar)**

### **Linear with MSE loss**

In [75]:
%matplotlib notebook
import tensorflow as tf
import keras as krs
import numpy as np
import pandas as pd

X = pd.read_csv('sonar.csv')

y = X.iloc[:,-1].astype('U20')
X = X.iloc[:,:-1].astype('float')

for i in range(len(y)):
    if y[i]=='Rock':
        y[i] = 0
    elif y[i]=='Mine':
        y[i] = 1
        
y = y.astype(int)

from sklearn.model_selection import train_test_split

xtrain, xtest, ytrain, ytest = train_test_split(X, y, test_size=0.25, random_state=0)

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.models import load_model
from keras.utils import np_utils

seed = 12345
numpy.random.seed(seed)

inputs_num = xtrain.shape[1]

ytrain = np_utils.to_categorical(ytrain)
ytest = np_utils.to_categorical(ytest)
num_classes = ytest.shape[1]
print(ytest.shape)

## Create our model
model = Sequential()

# 1st layer: RELU
model.add(Dense(500, input_dim=inputs_num, kernel_initializer='normal', activation='relu'))
# 2nd layer: sigmoid
model.add(Dense(100, input_dim=inputs_num, kernel_initializer='normal', activation='sigmoid'))
# output layer: activation linear
model.add(Dense(num_classes, kernel_initializer='normal', activation='linear'))

# Compile the model: MSE
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])

logger = keras.callbacks.ModelCheckpoint('sonar_model_MLP2HL_linear_mse.hdf5', monitor='val_acc', verbose=0, save_best_only=True)

model.fit(xtrain, ytrain, validation_data=(xtest, ytest), epochs=100, batch_size=200, verbose=2, callbacks=[logger])

scores = model.evaluate(xtest, ytest, verbose=0)
print("Test error: %.2f%%" % (100-scores[1]*100))

#reading the best model from file
model2 = load_model('sonar_model_MLP2HL_linear_mse.hdf5')
scores2 = model2.evaluate(xtest, ytest, batch_size=200)
print('network from file:')
print("Test error: %.2f%%" % (100-scores2[1]*100))

(52, 2)
Train on 155 samples, validate on 52 samples
Epoch 1/100
 - 5s - loss: 0.7115 - acc: 0.5419 - val_loss: 0.6082 - val_acc: 0.5192
Epoch 2/100
 - 0s - loss: 0.6049 - acc: 0.5419 - val_loss: 0.5136 - val_acc: 0.5192
Epoch 3/100
 - 0s - loss: 0.5111 - acc: 0.5419 - val_loss: 0.4321 - val_acc: 0.5192
Epoch 4/100
 - 0s - loss: 0.4300 - acc: 0.5419 - val_loss: 0.3642 - val_acc: 0.5192
Epoch 5/100
 - 0s - loss: 0.3621 - acc: 0.5419 - val_loss: 0.3110 - val_acc: 0.5192
Epoch 6/100
 - 0s - loss: 0.3086 - acc: 0.5419 - val_loss: 0.2740 - val_acc: 0.5192
Epoch 7/100
 - 0s - loss: 0.2706 - acc: 0.5419 - val_loss: 0.2541 - val_acc: 0.5192
Epoch 8/100
 - 0s - loss: 0.2491 - acc: 0.5419 - val_loss: 0.2509 - val_acc: 0.5192
Epoch 9/100
 - 0s - loss: 0.2437 - acc: 0.5419 - val_loss: 0.2613 - val_acc: 0.5577
Epoch 10/100
 - 0s - loss: 0.2514 - acc: 0.5548 - val_loss: 0.2782 - val_acc: 0.5577
Epoch 11/100
 - 0s - loss: 0.2656 - acc: 0.5677 - val_loss: 0.2936 - val_acc: 0.5577
Epoch 12/100
 - 0s - 

### **Softmax with cross-entropy loss**

In [76]:
%matplotlib notebook
import tensorflow as tf
import keras as krs
import numpy as np
import pandas as pd

X = pd.read_csv('sonar.csv')

y = X.iloc[:,-1].astype('U20')
X = X.iloc[:,:-1].astype('float')

for i in range(len(y)):
    if y[i]=='Rock':
        y[i] = 0
    elif y[i]=='Mine':
        y[i] = 1
        
y = y.astype(int)

from sklearn.model_selection import train_test_split

xtrain, xtest, ytrain, ytest = train_test_split(X, y, test_size=0.25, random_state=0)

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.models import load_model
from keras.utils import np_utils

seed = 12345
numpy.random.seed(seed)

inputs_num = xtrain.shape[1]

ytrain = np_utils.to_categorical(ytrain)
ytest = np_utils.to_categorical(ytest)
num_classes = ytest.shape[1]
print(ytest.shape)

## Create our model
model = Sequential()

# 1st layer: RELU
model.add(Dense(500, input_dim=inputs_num, kernel_initializer='normal', activation='relu'))
# 2nd layer: sigmoid
model.add(Dense(100, input_dim=inputs_num, kernel_initializer='normal', activation='sigmoid'))
# output layer: activation softmax
model.add(Dense(num_classes, kernel_initializer='normal', activation='softmax'))

# Compile the model: loss-crossentropy
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

logger = keras.callbacks.ModelCheckpoint('sonar_model_MLP2HL_softmax_cross.hdf5', monitor='val_acc', verbose=0, save_best_only=True)

model.fit(xtrain, ytrain, validation_data=(xtest, ytest), epochs=100, batch_size=200, verbose=2, callbacks=[logger])

scores = model.evaluate(xtest, ytest, verbose=0)
print("Test error: %.2f%%" % (100-scores[1]*100))

#reading the best model from file
model2 = load_model('sonar_model_MLP2HL_softmax_cross.hdf5')
scores2 = model2.evaluate(xtest, ytest, batch_size=200)
print('network from file:')
print("Test error: %.2f%%" % (100-scores2[1]*100))

(52, 2)
Train on 155 samples, validate on 52 samples
Epoch 1/100
 - 5s - loss: 0.6890 - acc: 0.5419 - val_loss: 0.6954 - val_acc: 0.5192
Epoch 2/100
 - 0s - loss: 0.6863 - acc: 0.5419 - val_loss: 0.6927 - val_acc: 0.5192
Epoch 3/100
 - 0s - loss: 0.6822 - acc: 0.5419 - val_loss: 0.6896 - val_acc: 0.5192
Epoch 4/100
 - 0s - loss: 0.6781 - acc: 0.5419 - val_loss: 0.6872 - val_acc: 0.5577
Epoch 5/100
 - 0s - loss: 0.6745 - acc: 0.5677 - val_loss: 0.6854 - val_acc: 0.5577
Epoch 6/100
 - 0s - loss: 0.6704 - acc: 0.6387 - val_loss: 0.6840 - val_acc: 0.5385
Epoch 7/100
 - 0s - loss: 0.6657 - acc: 0.6581 - val_loss: 0.6828 - val_acc: 0.5577
Epoch 8/100
 - 0s - loss: 0.6608 - acc: 0.6516 - val_loss: 0.6817 - val_acc: 0.5577
Epoch 9/100
 - 0s - loss: 0.6556 - acc: 0.6581 - val_loss: 0.6801 - val_acc: 0.5769
Epoch 10/100
 - 0s - loss: 0.6500 - acc: 0.6581 - val_loss: 0.6777 - val_acc: 0.5769
Epoch 11/100
 - 0s - loss: 0.6439 - acc: 0.6839 - val_loss: 0.6747 - val_acc: 0.5577
Epoch 12/100
 - 0s - 

# **Sonar: Comparing the results**

In [77]:
import tensorflow as tf
import numpy as np
import keras
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.models import load_model
from keras.utils import np_utils

X = pd.read_csv('sonar.csv')

y = X.iloc[:,-1].astype('U20')
X = X.iloc[:,:-1].astype('float')

for i in range(len(y)):
    if y[i]=='Rock':
        y[i] = 0
    elif y[i]=='Mine':
        y[i] = 1
        
y = y.astype(int)

from sklearn.model_selection import train_test_split

xtrain, xtest, ytrain, ytest = train_test_split(X, y, test_size=0.25, random_state=0)

# one hot encode outputs
ytrain = np_utils.to_categorical(ytrain)
ytest = np_utils.to_categorical(ytest)

#Compare the networks

#MLP1HL (Linear with MSE loss)
model = load_model('sonar_model_MLP1HL_linear_mse.hdf5')
scores = model.evaluate(xtest, ytest, batch_size=200)
print("Test error (Model MLP1HL (Linear with MSE loss)): %.2f%%" % (100-scores[1]*100))

#MLP1HL (Softmax with cross-entropy loss)
model = load_model('sonar_model_MLP1HL_softmax_cross.hdf5')
scores = model.evaluate(xtest, ytest, batch_size=200)
print("Test error (Model MLP1HL (Softmax with cross-entropy loss)): %.2f%%" % (100-scores[1]*100))

#MLP2HL (Linear with MSE loss)
model = load_model('sonar_model_MLP2HL_linear_mse.hdf5')
scores = model.evaluate(xtest, ytest, batch_size=200)
print("Test error (Model MLP2HL (Linear with MSE loss)): %.2f%%" % (100-scores[1]*100))

#MLP2HL (Softmax with cross-entropy loss)
model = load_model('sonar_model_MLP2HL_softmax_cross.hdf5')
scores = model.evaluate(xtest, ytest, batch_size=200)
print("Test error (Model MLP2HL (Softmax with cross-entropy loss)): %.2f%%" % (100-scores[1]*100))

print('end')

52/52 [==============================] - 2s 33ms/step
Test error (Model MLP1HL (Linear with MSE loss)): 15.38%
52/52 [==============================] - 2s 35ms/step
Test error (Model MLP1HL (Softmax with cross-entropy loss)): 15.38%
52/52 [==============================] - 2s 35ms/step
Test error (Model MLP2HL (Linear with MSE loss)): 15.38%
52/52 [==============================] - 2s 36ms/step
Test error (Model MLP2HL (Softmax with cross-entropy loss)): 15.38%
end
